<a href="https://colab.research.google.com/github/devyadav11/DL_assignment_2/blob/main/DLAssignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
from sklearn.utils import shuffle # for shuffling
import os
import cv2
import random
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image



from sklearn.model_selection import train_test_split

In [2]:
#!pip install wandb
import wandb
# !wandb login
wandb.login(key="500727e3b54d5202c75cd3136d6326f29e89e05f")

#key = 500727e3b54d5202c75cd3136d6326f29e89e05f

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ee23m074 (devyadav11) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
 !wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip


--2025-04-08 16:19:07--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.164.27, 172.217.0.91, 172.217.7.59, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.164.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G   111MB/s    in 50s     

2025-04-08 16:19:58 (72.1 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [4]:
 !unzip -q nature_12K.zip

In [5]:
 !rm nature_12K.zip

In [6]:
dtype = torch.float
device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)

cpu


In [7]:
# dataset_path = "/content/inaturalist_12K"

In [7]:
image_hight = 256
image_width = 256

In [15]:
def load_data(batchSize):
  transform_data = transforms.Compose([ transforms.Resize((image_hight, image_width)), transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ])   # normalise = output = (input - mean) / std ) (convert to (-1 to 1)

  train_data = datasets.ImageFolder(root = "/content/inaturalist_12K/train", transform=transform_data)

  class_index = train_data.class_to_idx

  training_idx = []
  val_idx  = []

  for train_class, train_index in class_index.items():
    index = []
    for k, (img_path, lable) in enumerate(train_data.samples):
      if lable == train_index:
        index.append(k)


    train_indices, val_indices = train_test_split(index, test_size=0.2, random_state=42)
    training_idx.extend(train_indices)           # extend creates a single flat list
    val_idx.extend(val_indices)

  training_data = Subset(train_data, training_idx)
  val_data = Subset(train_data, val_idx)


  #training_data, val_data = train_test_split(training_data, test_size=0.2, random_state=42)


  #train_loader = DataLoader(training_data, batch_size=batchSize, shuffle=True, pin_memory= True )
  #val_loader = DataLoader(val_data, batch_size=batchSize, shuffle=True, pin_memory= True)

  test_data = datasets.ImageFolder(root = "/content/inaturalist_12K/val", transform=transform_data)
  #test_laoder = DataLoader(test_data, batch_size=batchSize, shuffle=True, pin_memory= True)



  return training_data, val_data, test_data










  #return  class_index




In [ ]:
# transform = transforms.Compose([ transforms.Resize((256, 256)), transforms.ToTensor() ])
